# FD equations for the cogrowth sequence of $G=\mathbb{Z}_{n_1}^{*m_1}*\mathbb{Z}_{n_2}^{*m_2}*...*\mathbb{Z}_{n_k}^{*m_k}$, $n_1\ge 2$, each $m_i$ parameter or positive integer

Write $G=*_{i=1}^k *_{j=1}^{m_i} \langle x_{i,j} | x_{i,j}^{m_i} \rangle$.
Generating set is $S=\bigcup\{\{x_{i,j},x_{i,j}^{-1}\}: 1\le i \le k,1\le j \le m_i\}$. We denote $x_i$ as any $x_{i,j},j\le m_i$.

In [1]:
varstring ='m1 m2 m3 t'
var(varstring)
nis = [2,3,5]
dups = [m1,m2,m3]

In [2]:
def group_notation_latex():
    latexstr = []
    Z = r'\mathbb{Z}'
    for ni,mi in zip(nis,dups):
        latexstr.append('%s_{%s}'%(Z,ni)+('^{*%s}'%latex(mi) if mi!=1 else '') )
    return '*'.join(latexstr)

In [3]:
_lts = group_notation_latex()
print(_lts)
pretty_print('Generate Grammar for ',LatexExpr('\\,\\,'+_lts))

\mathbb{Z}_{2}^{*m_{1}}*\mathbb{Z}_{3}^{*m_{2}}*\mathbb{Z}_{5}^{*m_{3}}


'Generate Grammar for ' \,\,\mathbb{Z}_{2}^{*m_{1}}*\mathbb{Z}_{3}^{*m_{2}}*\mathbb{Z}_{5}^{*m_{3}}

In [4]:
## do not change!

params = [v for v in dups if type(v) not in [int,ZZ]]

k = len(nis)

# Compute free group and gen set
Fk = FreeGroup(k,['x%d'%i for i in (1..k)])

assert(list(map(str,Fk.gens()))==['x%d'%i for i in (1..k)]) # make sure that generators are named xi, i=1,...,k

xs = Fk.gens() # the gens xi=xs[i-1]
rel = []
for v,n in zip(xs,nis):
    rel.append(v^n)
G = Fk/ tuple(rel)

xs = G.gens()

In [5]:
Grs = G.rewriting_system() # a rewriting system to simplfy elements
Grs.make_confluent() # this guarrantees that equivalent elements map to a UNIQUE expression

def recmap(f):
    # f: an single argument function
    def _recmap_f(obj):
        # obj: atomic element or an iterable collection of them
        # return the reduced form for the element via a single argument function f,
        #    of the same iterable type, with each element recursively reduced
        try:
            iter(obj)
        except:
            # not iterable, so must be one atomic element
            return f(obj)
        container = type(obj)
        return container(list(map(_recmap_f, obj)))
    return _recmap_f

rd = recmap(Grs.reduce) # reduce elements of G, or iterable collections of them
Grs

Rewriting system of Finitely presented group < x1, x2, x3 | x1^2, x2^3, x3^5 >
with rules:
    x1^-1    --->    x1
    x1^2    --->    1
    x2^-2    --->    x2
    x2^2    --->    x2^-1
    x3^-3    --->    x3^2
    x3^3    --->    x3^-2

In [6]:
Sis = [Set(rd([xi,xi^-1])) for xi in xs]
print(Sis)

[{x1}, {x2, x2^-1}, {x3^-1, x3}]


In [7]:
S = set()
for A in Sis:
    S.update(A)
S= Set(S)
Sis,S

([{x1}, {x2, x2^-1}, {x3^-1, x3}], {x1, x2, x3^-1, x3, x2^-1})

In [8]:
# latex expression for sets
def set_to_latex(A):
    if A == Set():
        return r'\varnothing'
    return latex(A)

In [9]:
# define a space of variables indexed by the (g,X) pairs
# start with what we wish to solve, (1,\emptyset).
def series_expression(g,X):
    return 'F_{%s,%s}(t)'%(latex(g),set_to_latex(X))
var_space = {}
goal_pair = (G.one(), Set())
VAR_COUNTER = 0 # increment every time a new series variable is defined

def add_series_var(pair):
    if pair in var_space:
        return False
    global VAR_COUNTER
    g,X = pair
    var_space[pair] = var('v%d'%VAR_COUNTER, latex_name = series_expression(g,X))
    VAR_COUNTER += 1
    return True

def reset_var_space():
    global VAR_COUNTER
    var_space.clear()
    VAR_COUNTER=0
    add_series_var(goal_pair)
    
reset_var_space()

In [10]:
pretty_print(var_space)

{(1, {}): v0}

In [11]:
# need to define set-set, set-element multiplication
def set_mult(A,B):
    # turn any non-set object into a singleton set containing that object
    # assume elements all belong to G, and written in reduced from
    A,B = map(lambda S: S if S in Sets else Set([S]),[A,B])
    C= Set([a*b for a,b in cartesian_product([A,B])]) # assuming element multiplication is well defined
    return rd(C)

# Let's overload the '*' operator for Set objects
# Set is a function, but returns an instance of some spectifed kind of Set class, which is accessible via .parent()
reverse_if = lambda seq, cond: list(seq) if not cond else list(reversed(seq))
Set().parent()._mul_ =  lambda self, other, switch_sides=False: set_mult(*(reverse_if([self,other], switch_sides)))

In [12]:
# Now get the cyclic factors


def in_cyclic_factor(el, v):
    '''
        returns if the group element is in the given cyclic factor
        el: the element in G to check
        v: one of the xi's, the generator of the factor 
    '''
    expr = el.syllables()
    if not expr:
        return True
    return len(expr)==1 and expr[0][0] == v
    
def subset_of_cyclic_factor(A,v):
    # assume A is a finite sets
    return all((in_cyclic_factor(el,v) for el in A))

In [13]:
# construct the functional equation given the desired (g,X) pair
# assume {g}\cup X is completely in one of the cyclic factors
# CURRENTLY FINITE CASE ONLY

def get_FD_equation(pair, ret_new_pairs = True):
    '''
        returns a functional depency equation
        if ret_new_pairs is set to True, return a (equation, set of (g,X) pairs) pair instead
        assume g,X are rewritten in simplest form
    '''
    if ret_new_pairs:
        newpairs = set()
    g,X = pair
    eqn = None
    one = G.one()
    oneset = Set([one])
    if one not in X:
        XU1 = X.union(oneset)
        if add_series_var((one,X)) and ret_new_pairs:
            newpairs.add((one,X))
        if add_series_var((g,XU1)) and ret_new_pairs:
            newpairs.add((g,XU1))
        if g==one:
            eqn = 1+var_space[(one,X)]*(var_space[(one,XU1)]-1)
        else:
            eqn = var_space[(one,X)]*var_space[(g,XU1)]
    else:
        for v,gen in zip(xs, Sis):
            if subset_of_cyclic_factor(X,v):
                if g!=one:
                    eqn = t*ZZ(g in gen)
                    for s in gen-X:
                        [gs] = rd(s^-1)*Set([g]) # use set so auto reduction is implemented
                        Xs = rd(s^-1)*X
                        if add_series_var((gs,Xs)) and ret_new_pairs:
                            newpairs.add((gs,Xs))
                        eqn += t*var_space[(gs,Xs)]
                else:
                    eqn = 1*t^0
                    for gen_ext, nrep in zip(Sis,dups):
                        for s in gen_ext:
                            sinv = rd(s^-1)
                            set_sinv = Set([sinv])
                            if add_series_var((sinv,set_sinv)) and ret_new_pairs:
                                newpairs.add((sinv,set_sinv))
                            # need to subtract one when the gen set is within the same class of factors as {v}\cup X
                            eqn += t*var_space[(sinv,set_sinv)]*(nrep-ZZ(gen_ext==gen))
                    for s in gen-X:
                        [gs] = rd(s^-1)*Set([g]) # use set so auto reduction is implemented
                        Xs = rd(s^-1)*X
                        if add_series_var((gs,Xs)) and ret_new_pairs:
                            newpairs.add((gs,Xs))
                        eqn += t*var_space[(gs,Xs)]
                
                break
    eqn = var_space[pair]==eqn
    return (eqn,newpairs) if ret_new_pairs else eqn

In [14]:
# Let us build the system of equations


reset_var_space()

system = []
queue = [goal_pair]

# use BFS
while queue:
    pair = queue.pop(0)
    #pretty_print(pair)
    eqn, newp = get_FD_equation(pair)
    queue.extend(newp)
    system.append(eqn)

In [15]:
#Here are the equations
for eqn in system:
    print(latex(eqn),r'\\')
print()
print()
for eqn in system:
    #print(eqn)
    pretty_print(eqn)

{F_{1,\varnothing}(t)} = {F_{1,\varnothing}(t)} {\left({F_{1,\left\{1\right\}}(t)} - 1\right)} + 1 \\
{F_{1,\left\{1\right\}}(t)} = {\left(m_{1} - 1\right)} t {F_{x_{1},\left\{x_{1}\right\}}(t)} + m_{2} t {F_{x_{2}^{-1},\left\{x_{2}^{-1}\right\}}(t)} + m_{2} t {F_{x_{2},\left\{x_{2}\right\}}(t)} + m_{3} t {F_{x_{3},\left\{x_{3}\right\}}(t)} + m_{3} t {F_{x_{3}^{-1},\left\{x_{3}^{-1}\right\}}(t)} + t {F_{x_{1},\left\{x_{1}\right\}}(t)} + 1 \\
{F_{x_{2},\left\{x_{2}\right\}}(t)} = {F_{1,\left\{x_{2}\right\}}(t)} {F_{x_{2},\left\{x_{2}, 1\right\}}(t)} \\
{F_{x_{3},\left\{x_{3}\right\}}(t)} = {F_{x_{3},\left\{1, x_{3}\right\}}(t)} {F_{1,\left\{x_{3}\right\}}(t)} \\
{F_{x_{3}^{-1},\left\{x_{3}^{-1}\right\}}(t)} = {F_{1,\left\{x_{3}^{-1}\right\}}(t)} {F_{x_{3}^{-1},\left\{x_{3}^{-1}, 1\right\}}(t)} \\
{F_{x_{1},\left\{x_{1}\right\}}(t)} = {F_{1,\left\{x_{1}\right\}}(t)} {F_{x_{1},\left\{1, x_{1}\right\}}(t)} \\
{F_{x_{2}^{-1},\left\{x_{2}^{-1}\right\}}(t)} = {F_{1,\left\{x_{2}^{-1}\right\}}(

v0 == v0*(v1 - 1) + 1

v1 == (m1 - 1)*t*v2 + m2*t*v3 + m2*t*v4 + m3*t*v5 + m3*t*v6 + t*v2 + 1

v4 == v7*v8

v5 == v10*v9

...

v61 == v59*v65

v62 == t

v63 == 0

v64 == m1*t*v2 + m2*t*v3 + m2*t*v4 + (m3 - 1)*t*v5 + (m3 - 1)*t*v6 + 1

v65 == 0

In [16]:
# There are symmetries amongst the variables such that some variables will repersent the same series.
# Amongst each such equivalence class, we only need to keep one representation.
# To strip symmetry, we need union find.

from collections import defaultdict as DD

class UF(object):
    
    def __init__(self):
        self.V = {}
        self.parent = {}
    
    def add(self, node):
        if node not in self.V:
            self.V[node] = 1
            self.parent[node]=node
    
    def find(self,u):
        P = self.parent
        while P[u]!=u:
            P[u] = P[P[u]]
            u = P[u]
        return u
    
    def union(self,u,v):
        u,v = map(self.find, [u,v])
        u,v = sorted([u,v],key = lambda s: self.V[s])
        self.parent[u]=v
        self.V[v]+=self.V[u]
    
    def verts(self):
        return iter(self.V)
    
    def comps(self):
        ans = DD(set)
        for v in self.V:
            ans[self.find(v)].add(v)
        return ans
    


In [17]:
# initialize the structure to track symmetry
eqcl = UF()
for pair in var_space:
    eqcl.add(pair)

#eqcl.comps()

In [18]:
# since S is inverse closed, then (g,X) is essentially the same as (g^-1,X^-1)

vis = set()
for g,X in var_space:
    if (g,X) in vis:
        continue
    ginv = rd(g^-1)
    Xinv = Set([v^-1 for v in X])
    Xinv = rd(Xinv)
    pairinv = (ginv,Xinv)
    if pairinv in var_space:
        vis.add(pairinv)
        eqcl.union((g,X),pairinv)
#eqcl.comps()

In [19]:
from pprint import pprint
pprint(eqcl.comps())

defaultdict(<class 'set'>,
            {(1, {1, x3^-2, x3^2, x3^-1, x3}): {(1,
                                                 {1, x3^-2, x3^2, x3^-1, x3})},
             (1, {x3^-1, x3, x3^-2, x3^2}): {(1, {x3^-1, x3, x3^-2, x3^2})},
             (1, {x3^-1, 1, x3^-2, x3^2}): {(1, {x3^-1, 1, x3^-2, x3^2}),
                                            (1, {1, x3, x3^-2, x3^2})},
             (1, {x3^-1, x3^-2, x3^2}): {(1, {x3, x3^-2, x3^2}),
                                         (1, {x3^-1, x3^-2, x3^2})},
             (1, {x3^-1, 1, x3^-2}): {(1, {1, x3, x3^2}),
                                      (1, {x3^-1, 1, x3^-2})},
             (1, {x2, 1, x2^-1}): {(1, {x2, 1, x2^-1})},
             (1, {x3^-1, x3^-2}): {(1, {x3, x3^2}), (1, {x3^-1, x3^-2})},
             (1, {x2, x2^-1}): {(1, {x2, x2^-1})},
             (1, {1, x1}): {(1, {1, x1})},
             (1, {x3^-1, 1}): {(1, {x3^-1, 1}), (1, {1, x3})},
             (1, {1, x2^-1}): {(1, {x2, 1}), (1, {1, x2^-1})},
            

In [20]:
# now make subtitutions to the system based on the symetries
reduced_system = []
rev_var_space = {j:i for i,j in var_space.items()}
for eq in system:
    v = eq.lhs()
    pv = rev_var_space[v]
    if pv!=eqcl.find(pv):
        continue
    ag = eq.rhs().args()
    ag = set(ag)-{t}-set(params)
    ag = list(ag)
    agrepl = [rev_var_space[u] for u in ag]
    agrepl = [eqcl.find(u) for u in agrepl]
    agrepl = [var_space[u] for u in agrepl]
    reduced_system.append(v==(eq.rhs().subs(dict(zip(ag,agrepl)) )).expand().simplify())

In [21]:
for eq in reduced_system:
    pretty_print(eq)

v0 == v0*v1 - v0 + 1

v1 == m1*t*v2 + 2*m2*t*v4 + 2*m3*t*v6 + 1

v4 == v15*v16

v6 == v11*v12

v2 == v13*v14

...

v59 == v59*v64 - v59 + 1

v61 == v59*v65

v62 == t

v64 == m1*t*v2 + 2*m2*t*v4 + 2*m3*t*v6 - 2*t*v6 + 1

v65 == 0

In [22]:
# get rid of all F-varibles that only depend on t
while 1:
    terminals = {}
    for eq in reduced_system:
        if Set(eq.rhs().args()).issubset(Set([t])):
            terminals[eq.lhs()] = eq.rhs()
    if not terminals:
        break
    reduced_system = [eq.subs(terminals) for eq in reduced_system if eq.lhs() not in terminals]

In [23]:
for eq in reduced_system:
    pretty_print(eq)

v0 == v0*v1 - v0 + 1

v1 == m1*t*v2 + 2*m2*t*v4 + 2*m3*t*v6 + 1

v4 == v15*v16

v6 == v11*v12

v2 == t*v13

...

v47 == v47*v56 - v47 + 1

v55 == t*v59

v56 == m1*t*v2 + 2*m2*t*v4 + 2*m3*t*v6 + t*v55 - 2*t*v6 + 1

v59 == v59*v64 - v59 + 1

v64 == m1*t*v2 + 2*m2*t*v4 + 2*m3*t*v6 - 2*t*v6 + 1

In [24]:
# Take the reduced implicit system of equations, with the list of variables to eliminate,
#  and dump them to a maple data file
import os

var_rem = {eq.lhs() for eq in reduced_system}
print(os.getcwd())
fdir = 'maple_data_files'
if not os.path.isdir(fdir):
    os.mkdir(fdir)
fsuffix = '-dat.maple'
fname = ''
for n,rep in zip(nis,dups):
    fname+='Z%d_%s__'%(n,rep)
fname+='ic'
fname+=fsuffix
fname = os.path.join(fdir,fname)
if not os.path.exists(fname):
    fd = open(fname, 'w')
    fd.write('syst:='+str(reduced_system).replace('==','=')+';')
    fd.write('\n')
    fd.write('varlist:='+str((sorted(var_rem, key = lambda vv: int(str(vv)[1:]))))+';')
    fd.write('\n')
    fd.close()

/home/sage/OneDrive - sfu.ca/Research/Msc_math_sfu/Marni/cogrowth_computations


In [25]:
# check if new file exists
print(fname,os.path.exists(fname))

maple_data_files/Z2_m1__Z3_m2__Z5_m3__ic-dat.maple True


In [28]:
# Take the latex code of the system, and dump it to a file
fdir = 'grammar_latex'
if not os.path.isdir(fdir):
    os.mkdir(fdir)
fsuffix = 'grammar.txt'
fname = ''
for n,rep in zip(nis,dups):
    fname+='Z%d_%s__'%(n,rep)
fname += fsuffix
fname = os.path.join(fdir,fname)
if not os.path.exists(fname):
    fd = open(fname, 'w')
    for eqn in reduced_system:
        fd.write(latex(eqn)+"\\\\\n")
    fd.close()

